In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/real-estate-dataset/data.csv")
pd.set_option("display.max_columns", 1000)
df.head()

In [ ]:
df['NOX'].describe()

In [ ]:
df['AGE'].describe()

In [ ]:
y = df['CRIM']
X = df.drop("CRIM" , axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor()

grid = {'learning_rate': list(np.arange(0.03 , 0.15 , 0.02)),
        'depth': [3 , 4 , 5 ],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid, 
                                       X=X_train, 
                                       y=y_train, 
                                       cv = 5,
                                       plot=True)

In [ ]:
grid_search_result['params']

In [ ]:
model = CatBoostRegressor(iterations=1000,
                        grid_search_result['params'].depth = 3 ,
                         grid_search_result['params'].l2_leaf_reg = 9 , 
                         grid_search_result['params'].learning_rate= 0.11)
# Fit model
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
print(mean_absolute_error(y_test, y_pred))
print(mean_absolute_percentage_error(y_test, y_pred))
print(explained_variance_score(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
submission = pd.DataFrame(None)
# submission['pred']= y_pred
submission['CRIM']= y_test

submission['error']= abs(y_pred - y_test)


In [ ]:
submission.to_csv("submission.csv")